Original notebook: https://colab.research.google.com/drive/1iomGvGlYzvNHhTlGcFbOD1wwpZr5HT89?usp=sharing

Below is not a direct copy, but a version adapted to work without Google Drive mounting / colab and for **one** run of the `downloadHop1Links.py`. This notebook is not meant to compare files.

In [25]:
import pandas as pd
import json
import ast

errorFileName = 'generalErrorUniversities-2024-07-22-03' # Should NOT contain file extension, used elsewhere in the code to create new files
storeLocation = './helperFiles/errorAnalysisOneRun/'

dfError = pd.read_csv(f'../errorLoggingOneHop/localRun/{errorFileName}.csv')

dfError.head(3)

,name,link,error
0,River Parishes Community College,https://www.rpcc.edu/#main,HTTP Error 403: Forbidden
1,River Parishes Community College,https://www.rpcc.edu/,HTTP Error 403: Forbidden
2,River Parishes Community College,https://www.rpcc.edu/#,HTTP Error 403: Forbidden


In [26]:
dfError.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124308 entries, 0 to 124307
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   name    124308 non-null  object
 1   link    124308 non-null  object
 2   error   124240 non-null  object
dtypes: object(3)
memory usage: 2.8+ MB


In [27]:
dfError.head()

,name,link,error
0,River Parishes Community College,https://www.rpcc.edu/#main,HTTP Error 403: Forbidden
1,River Parishes Community College,https://www.rpcc.edu/,HTTP Error 403: Forbidden
2,River Parishes Community College,https://www.rpcc.edu/#,HTTP Error 403: Forbidden
3,River Parishes Community College,https://www.rpcc.edu/about-us,HTTP Error 403: Forbidden
4,River Parishes Community College,https://www.rpcc.edu/campuses/index,HTTP Error 403: Forbidden


In [28]:
dfError.error.value_counts()[0:20]

HTTP Error 403: Forbidden                                                                                                                 75278
HTTP Error 404: Not Found                                                                                                                 14136
[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)                           8584
HTTP Error 400: Bad Request                                                                                                                4776
'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte                                                             3990
HTTP Error 405: Not Allowed                                                                                                                2326
HTTP Error 429: Too Many Requests                                                                                                       

In [29]:
dfError.error.value_counts()

HTTP Error 403: Forbidden                                                                                           75278
HTTP Error 404: Not Found                                                                                           14136
[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1129)     8584
HTTP Error 400: Bad Request                                                                                          4776
'utf-8' codec can't decode byte 0xe2 in position 10: invalid continuation byte                                       3990
                                                                                                                    ...  
URL can't contain control characters. '/downloads/registrar/course list/Past/FA16.pdf' (found at least ' ')             1
'utf-8' codec can't decode byte 0xc2 in position 4: invalid continuation byte                                           1
'utf-8' codec can't deco

In [30]:
filterColumns = ['name', 'error']
dfFiltered = dfError.drop_duplicates(subset = filterColumns)
dfFiltered.to_csv(f'{storeLocation}{errorFileName}--Filtered.csv', index=False)
dfFiltered.head()

,name,link,error
0,River Parishes Community College,https://www.rpcc.edu/#main,HTTP Error 403: Forbidden
14,Lower Columbia College,https://services4.lowercolumbia.edu/employees/,[SSL: CERTIFICATE_VERIFY_FAILED] certificate v...
16,Bryant & Stratton College-Albany,https://www.bryantstratton.edu/student-experie...,HTTP Error 403: Forbidden
17,Princeton University,https://www.princeton.edu,HTTP Error 403: Forbidden
60,Great Basin College,https://www.gbcnv.edu/javascript:void(0),HTTP Error 404: Not Found


In [31]:
# Convert value counts / frequency counts to percentages

dfFilt_2 = pd.read_csv(f'{storeLocation}{errorFileName}--Filtered.csv')
totalRows = len(dfFilt_2)
totalNumPerValue = dfFilt_2['error'].value_counts()
percent = (totalNumPerValue / totalRows) * 100

dfPercent = percent.reset_index()
dfPercent.columns = ['error', 'frequency (percentage)']
dfPercent['frequency (percentage)'] = dfPercent['frequency (percentage)'].round(6)

dfPercent.to_csv(f'{storeLocation}{errorFileName}--Filtered--Percent--Rounded.csv', index=False)
dfPercent.head(5)

,error,frequency (percentage)
0,HTTP Error 404: Not Found,15.443012
1,HTTP Error 403: Forbidden,15.141431
2,HTTP Error 999: Request denied,10.700915
3,'utf-8' codec can't decode byte 0xe2 in positi...,5.043677
4,HTTP Error 429: Too Many Requests,3.161398


In [32]:
# Lists all errors for each university with multiple errors
repeatedUniversities = dfFilt_2['name'][dfFilt_2['name'].duplicated(keep=False)]
dfUni_1 = dfFilt_2[dfFilt_2['name'].isin(repeatedUniversities)]
dfUni_Final = dfUni_1.groupby('name').agg({'error': lambda x: ','.join(x.astype(str))}).reset_index()

dfUni_Final.to_csv(f'{storeLocation}{errorFileName}--Filtered--Universities--Multiple_Errors.csv', index=False)
dfUni_Final.head()

,name,error
0,Aaniiih Nakoda College,'utf-8' codec can't decode byte 0xe2 in positi...
1,Abilene Christian University,"HTTP Error 403: Forbidden,HTTP Error 404: Not ..."
2,Abraham Baldwin Agricultural College,"HTTP Error 405: Not Allowed,HTTP Error 429: To..."
3,Abraham Lincoln University,"HTTP Error 404: Not Found,'utf-8' codec can't ..."
4,Academy College,"HTTP Error 403: Forbidden,HTTP Error 404: Not ..."


In [33]:
# Group utf-8 errors

# parse dfFilt_2
# or actually parse dfPercent and group utfs
column_length = len(dfFilt_2)
utfValues = dfFilt_2['error'].str.startswith('\'utf-8\'')
totalUTF = utfValues.sum()

percent = (totalUTF / totalRows) * 100
# percent = percent.round(6)
print(percent)

dfUTF = dfPercent
utfPercentRow = {'error': 'totalUTF', 'frequency (percentage)': percent}
utf_temp_df = pd.DataFrame([utfPercentRow])
dfUTF = pd.concat([dfUTF, utf_temp_df], ignore_index = True)
dfUTF.to_csv(f'{storeLocation}{errorFileName}--Filtered--Percent--UTF_Total.csv', index=False)

15.016638935108153


In [34]:
# Group URL can't contain errors

# parse dfFilt_2
# or actually parse dfPercent and group utfs
column_length = len(dfFilt_2)
controlValues = dfFilt_2['error'].str.startswith('URL can\'t contain control characters')
totalControlUTF = controlValues.sum()

controlPercent = (totalControlUTF / totalRows) * 100
# controlPercent = controlPercent.round(6)
print(controlPercent)

16.618136439267886


In [35]:
dfGrouped = dfPercent[~dfPercent.error.str.contains("utf-8")].reset_index(drop=True)
dfGrouped = dfGrouped[~dfGrouped.error.str.contains("URL can\'t contain control characters")].reset_index(drop=True)

dfExtra = pd.DataFrame([{'error':"GROUPED 'utf-8' codec can't decode byte", 'frequency (percentage)': percent}, {'error':"GROUPED URL can\'t contain control characters", 'frequency (percentage)': controlPercent}])

dfAllGrouped = pd.concat([dfExtra, dfGrouped], ignore_index=True)

dfAllGrouped = dfAllGrouped.sort_values(by=['frequency (percentage)'], ascending=False)

dfAllGrouped.head(5)


,error,frequency (percentage)
1,GROUPED URL can't contain control characters,16.618136
2,HTTP Error 404: Not Found,15.443012
3,HTTP Error 403: Forbidden,15.141431
0,GROUPED 'utf-8' codec can't decode byte,15.016639
4,HTTP Error 999: Request denied,10.700915


In [36]:
dfAllGrouped.to_csv(f'{storeLocation}{errorFileName}--Filtered--Universities--Grouped_Errors.csv', index=False)